In [ ]:
# This notebook demonstrates how to calculate 3D Zernike moments using ZMPY3D with TensorFlow.
#
# This notebook primarily consists of the following steps: 
#     1. Install ZMPY3D_TF.
#     2. Define necessary parameters.
#     3. Load precalculated cache.
#     4. Download example PDB data with coordinates.
#     5. Convert coordinate data into a voxel.
#     6. Create a callable function for generating Zernike moments and normalization.
#     7. Obtain the results.
#     8. A command line interface (CLI) example

In [ ]:
# Install ZMPY3D versions for TensorFlow.
! pip install ZMPY3D-TF
print(f"It is recommended to restart the Python kernel for the IPython notebook.")



In [ ]:
# Download example data from GitHub using curl
! curl -OJL https://github.com/tawssie/ZMPY3D/raw/main/1WAC_A.txt


In [ ]:

import ZMPY3D_TF as z
import tensorflow as tf
import pickle

MaxOrder = 20 # Set the maximum order to 20
GridWidth= 1.00 # Set the grid width to 1.00 (equivalent to 1 angstrom)
Param=z.get_global_parameter() # Retrieve the necessary parameters

# Find the cache_data directory based on the site package location of ZMPY3D.
LogCacheFilePath=z.__file__.replace('__init__.py', 'cache_data') + '/LogG_CLMCache_MaxOrder{:02d}.pkl'.format(MaxOrder)

with open(LogCacheFilePath, 'rb') as file:
    CachePKL = pickle.load(file)

# Convert the precalculated cache data into TensorFlow tensor objects
GCache_pqr_linear= tf.convert_to_tensor(CachePKL['GCache_pqr_linear'])
GCache_complex= tf.convert_to_tensor(CachePKL['GCache_complex'])
GCache_complex_index= tf.convert_to_tensor(CachePKL['GCache_complex_index'])
CLMCache3D= tf.convert_to_tensor(CachePKL['CLMCache3D'],dtype=tf.complex128)
CLMCache= tf.convert_to_tensor(CachePKL['CLMCache'], dtype=tf.float64)


print(f"Now using the MaxOrder of {MaxOrder} and the GridWidth of {GridWidth}.")
print(f"Pre-calculated parameters have been loaded successfully.")


In [ ]:
%%time
PDBFileName='./1WAC_A.txt'


# Convert structure data into coordinates
[XYZ,AA_NameList]=z.get_pdb_xyz_ca(PDBFileName)

# Convert coordinates into voxels using precalculated Gaussian densities
ResidueBox=z.get_residue_gaussian_density_cache(Param)
[Voxel3D,Corner]=z.fill_voxel_by_weight_density(XYZ,AA_NameList,Param['residue_weight_map'],GridWidth,ResidueBox[GridWidth])

# Convert the voxel data into a tensor object
Voxel3D=tf.convert_to_tensor(Voxel3D,dtype=tf.float64)

print(f"Converting PDB to 3D voxel grid with NumPy on CPU, then transferring to GPU memory as Tensorflow objects.")
print(f"Time elapsed is as follows:")

In [ ]:
%%time

Dimension_BBox_scaled=tf.shape(Voxel3D)

MaxOrder=tf.convert_to_tensor(MaxOrder,dtype=tf.int64)

X_sample = tf.range(Dimension_BBox_scaled[0] + 1,dtype=tf.float64)
Y_sample = tf.range(Dimension_BBox_scaled[1] + 1,dtype=tf.float64)
Z_sample = tf.range(Dimension_BBox_scaled[2] + 1,dtype=tf.float64)

# Calculate the volume mass and the center of mass
[VolumeMass,Center,_]=z.calculate_bbox_moment(Voxel3D,1,X_sample,Y_sample,Z_sample)

# Calculate the weights for sphere sampling
[AverageVoxelDist2Center,_]=z.calculate_molecular_radius(Voxel3D,Center,VolumeMass,tf.convert_to_tensor(Param['default_radius_multiplier'], dtype=tf.float64))
Sphere_X_sample, Sphere_Y_sample, Sphere_Z_sample=z.get_bbox_moment_xyz_sample(Center,AverageVoxelDist2Center,Dimension_BBox_scaled)

# Apply weights to the geometric moments
_,_,SphereBBoxMoment=z.calculate_bbox_moment(Voxel3D
                                  ,MaxOrder
                                  ,Sphere_X_sample
                                  ,Sphere_Y_sample
                                  ,Sphere_Z_sample)

# Convert to scaled 3D Zernike moments
ZMoment_scaled,_=z.calculate_bbox_moment_2_zm(MaxOrder
                                   , GCache_complex
                                   , GCache_pqr_linear
                                   , GCache_complex_index
                                   , CLMCache3D
                                   , SphereBBoxMoment)

# Convert the scaled 3D Zernike moments into 3DZD-based descriptors
ZM_3DZD_invariant=z.get_3dzd_121_descriptor(ZMoment_scaled)
ZM_3DZD_invariant_121=tf.reshape(tf.boolean_mask(ZM_3DZD_invariant, ~tf.math.is_nan(ZM_3DZD_invariant)), [-1])


print(f"The dimensions of the voxel being used are {Voxel3D.shape}.")
print(f"Transforming the 3D voxel into Zernike moments with global normalization, 3DZD style, yields 121 descriptors.")
print(f"Time elapsed is as follows:")

In [ ]:
@tf.function
def OneTimeConversion_TF(Voxel3D,MaxOrder):
    Dimension_BBox_scaled=tf.shape(Voxel3D)
    MaxOrder=tf.convert_to_tensor(MaxOrder,dtype=tf.int64)
    
    X_sample = tf.range(Dimension_BBox_scaled[0] + 1,dtype=tf.float64)
    Y_sample = tf.range(Dimension_BBox_scaled[1] + 1,dtype=tf.float64)
    Z_sample = tf.range(Dimension_BBox_scaled[2] + 1,dtype=tf.float64)
    
    [VolumeMass,Center,_]=z.calculate_bbox_moment(Voxel3D,1,X_sample,Y_sample,Z_sample)
    [AverageVoxelDist2Center,_]=z.calculate_molecular_radius(Voxel3D,Center,VolumeMass,tf.convert_to_tensor(Param['default_radius_multiplier'], dtype=tf.float64))
    Sphere_X_sample, Sphere_Y_sample, Sphere_Z_sample=z.get_bbox_moment_xyz_sample(Center,AverageVoxelDist2Center,Dimension_BBox_scaled)
    
    _,_,SphereBBoxMoment=z.calculate_bbox_moment(Voxel3D
                                      ,MaxOrder
                                      ,Sphere_X_sample
                                      ,Sphere_Y_sample
                                      ,Sphere_Z_sample)
    
    ZMoment_scaled,_=z.calculate_bbox_moment_2_zm(MaxOrder
                                       , GCache_complex
                                       , GCache_pqr_linear
                                       , GCache_complex_index
                                       , CLMCache3D
                                       , SphereBBoxMoment)
    
    ZM_3DZD_invariant=z.get_3dzd_121_descriptor(ZMoment_scaled)
    
    ZM_3DZD_invariant_121=tf.reshape(tf.boolean_mask(ZM_3DZD_invariant, ~tf.math.is_nan(ZM_3DZD_invariant)), [-1])
    return ZM_3DZD_invariant_121

print(f"Merge all steps into a single callable Tensorfow function, OneTimeConversion_TF, decorated with @tf.function for optimized graph compilation.")


In [ ]:
%%time
# Use OneTimeConversion_TF with @tf.function to calculate and normalize 3D Zernike moments
print(OneTimeConversion_TF(Voxel3D,MaxOrder))

In [ ]:

# Alternatively, use a system call to compute results via CLI
# ./ZMPY3D_TF_CLI_ZM PDBFile GridWidth MaximumOrder NormOrder Mode
! ZMPY3D_TF_CLI_ZM "./1WAC_A.txt" 1.0 20 2 1

